In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv
/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv
/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv


In [2]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import re
import gc
import numpy as np
import math
import catboost as cb
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_squared_log_error,make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,RidgeCV,BayesianRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import math
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud, STOPWORDS
stopwords1 = set(STOPWORDS)
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy.sparse import csr_matrix
from scipy import sparse
import scipy
from sklearn.linear_model import Ridge
%matplotlib inline

In [3]:
train = pd.read_csv("/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv")
test = pd.read_csv("/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv")
sample = pd.read_csv("/kaggle/input/buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv")

In [4]:
train.shape,test.shape

((5429, 9), (2327, 8))

In [5]:
dataset = pd.concat([train ,test])
dataset = dataset.reset_index(drop=True)
dataset = dataset.sort_values(by=['session_id'])

In [6]:
train['client_agent'][0]

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.111 Safari/537.36'

In [7]:
dataset["browser_version"] =dataset["client_agent"].str.split("/",expand=True)[1].str.split(" ",expand=True)[0]

In [8]:
dataset [["device","browser"]] =dataset["device_details"].str.split(pat=" - ",expand=True)

In [9]:
handheld = [int(i in ["iPhone","Android Phone","Android Tablet","iPad"]) for i in dataset ["device"]]
dataset ["hand_held"] = pd.Series(handheld)

desktop = [int(i == "Desktop") for i in dataset ["device"]]
dataset ["desktop"] = pd.Series(desktop)

others = [int(i in ["Other","Unknown"]) for i in dataset["device"]]
dataset["others"] = pd.Series(others)

In [10]:
dataset['date']= pd.to_datetime(dataset['date'])

dataset["month"] = dataset['date'].dt.month
dataset["year"] = dataset['date'].dt.year
dataset["week"] = dataset['date'].dt.isocalendar().week
dataset["dayOfWeek"] = dataset['date'].dt.dayofweek
dataset["day"] = dataset["date"].dt.day

In [11]:
dataset["dailyTraffic"] = dataset.groupby("date")["session_id"].transform("count")
dataset["dailyTrafficPerDevice"] = dataset.groupby(["date","device"])["session_id"].transform("count")
dataset["dailyTrafficPerbrowser"] = dataset.groupby(["date","browser"])["session_id"].transform("count")


dataset["dailyAverageTime"] = dataset.groupby("date")["time_spent"].transform("mean")
dataset["meanTimePerDevice"] = dataset.groupby("device")["time_spent"].transform("mean")
dataset["meanTimePerBrowser"] = dataset.groupby("browser")["time_spent"].transform("mean")

In [12]:
dataset["device_num"] = dataset["device"].replace({"Other": 1, "Unknown": 1, 
                         "Android Tablet":2,
                        "iPhone":3,
                        "Android Phone":4,
                        "iPad":5, "Desktop":5})

/tmp/ipykernel_17/2519709877.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset["device_num"] = dataset["device"].replace({"Other": 1, "Unknown": 1,


In [13]:
dataset["browser_version"] = dataset["browser_version"].fillna('n/a')

In [14]:
dataset=pd.get_dummies(dataset,columns=['device_num','browser_version'],drop_first=True)

In [15]:
fin_features = list(set(dataset.columns) - set(["session_id","client_agent","date","device","browser","device_details","time_spent","index"]))
fin_features

['browser_version_5.0',
 'browser_version_4.4.1',
 'device_num_5',
 'browser_version_2.1.0',
 'browser_version_4.6.0',
 'dailyTrafficPerDevice',
 'browser_version_6533.21.1',
 'browser_version_160391',
 'browser_version_150264',
 'browser_version_4.7.0',
 'browser_version_150302',
 'device_num_4',
 'browser_version_319',
 'browser_version_160341',
 'browser_version_160355',
 'browser_version_3.1.0',
 'browser_version_2.5.1',
 'checked_out',
 'device_num_3',
 'browser_version_160320',
 'browser_version_160399',
 'browser_version_4.5.1',
 'week',
 'browser_version_160314',
 'session_number',
 'browser_version_160360',
 'browser_version_160418',
 'browser_version_4.3.3',
 'browser_version_150259',
 'browser_version_160432',
 'browser_version_160375',
 'month',
 'dayOfWeek',
 'browser_version_160362',
 'browser_version_160387',
 'dailyAverageTime',
 'browser_version_245',
 'browser_version_3.5.1',
 'browser_version_160326',
 'browser_version_160318',
 'browser_version_160338',
 'hand_held'

In [16]:
dataset["client_agent"] = dataset["client_agent"].fillna('n/a')

In [17]:
dftrain, dftest  = dataset[dataset["time_spent"].isnull()==False], dataset[dataset["time_spent"].isnull()==True]
dftest.reset_index(drop=True, inplace=True)
target = np.log1p(dftrain['time_spent'])

In [18]:
v_1 = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
typ_tr =v_1.fit_transform(dftrain['client_agent'])
typ_ts =v_1.transform(dftest['client_agent'])

In [19]:
v_1c = TfidfVectorizer(ngram_range=(2,6),stop_words="english", analyzer='char')
typ_trc =v_1c.fit_transform(dftrain['client_agent'])
typ_tsc =v_1c.transform(dftest['client_agent'])

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [20]:
final_features = sparse.hstack((dftrain[fin_features].astype(float),typ_tr,typ_trc)).tocsr()
final_featurest = sparse.hstack((dftest[fin_features].astype(float),typ_ts ,typ_tsc)).tocsr()

In [21]:
def rmsle(real, predicted):
    real=np.expm1(real)
    predicted=np.expm1(predicted)
    return np.sqrt(mean_squared_log_error(real,predicted))
    

In [22]:
X_trn, X_val, y_trn, y_val = train_test_split(final_features, target, test_size=0.25, random_state=1996)
X_test = final_featurest

In [23]:
cb_params={'depth':[2,3],'learning_rate':np.linspace(0.0001,0.001,15),'iterations':[3,4,5,6]}

In [24]:
#cb_grid=GridSearchCV(estimator=cb.CatBoostRegressor(verbose=0),param_grid=cb_params,scoring=rmsle,cv=3,verbose=2)

In [25]:
#cb_grid.fit(X_trn,y_trn)

In [26]:
#cb_grid.best_params_

In [27]:
cb_model=cb.CatBoostRegressor(depth=2,iterations=3,learning_rate=0.00001,verbose=0)

In [28]:
cb_model.fit(X_trn,y_trn)

In [29]:
cb_preds=cb_model.predict(X_val)

In [30]:
np.sqrt(mean_squared_log_error(np.expm1(y_val.values),np.expm1(cb_preds)))

1.7531379857234903

In [31]:
cb_preds=cb_model.predict(X_test)

In [32]:
cb_preds_df=pd.DataFrame(data=np.expm1(cb_preds),columns=['time_spent'])

In [33]:
cb_preds_df.to_csv('Sample Submission.csv',index=False)